In [4]:
# Libraries
import requests
import json
import pandas as pd

In [6]:
# load in prior data set

data = pd.read_excel("/Users/adammcmahan/milestone4.xlsx")
print(data.head())

# load in API and set

file_path = "/Users/adammcmahan/Desktop/540 Project API.json"

with open(file_path) as f:
    keys = json.load(f)
    api_key = keys["apikey"] 

url = "http://www.omdbapi.com/"
api_variable = f"&apikey={api_key}"

   id     type                                             title  \
0   1    Movie  Duck the Halls: A Mickey Mouse Christmas Special   
1   2    Movie                            Ernest Saves Christmas   
2   3    Movie                      Ice Age: A Mammoth Christmas   
3   4    Movie                        The Queen Family Singalong   
4   5  TV Show                             The Beatles: Get Back   

                            director  \
0  Alonso Ramirez Ramos, Dave Wasson   
1                        John Cherry   
2                       Karen Disher   
3                    Hamish Hamilton   
4                                NaN   

                                                cast        country  \
0  Chris Diamantopoulos, Tony Anselmo, Tress MacN...            NaN   
1           Jim Varney, Noelle Parker, Douglas Seale            NaN   
2  Raymond Albert Romano, John Leguizamo, Denis L...  United States   
3  Darren Criss, Adam Lambert, Derek Hough, Alexa...            Na

In [7]:
# Define movie titles or IDs to fetch new data
movie_list = [
    "Wings", "The Racket", "7th Heaven", "Sunrise", "The Last Command",
    "Street Angel", "Tempest", "The Dove", "The Way of All Flesh", "Two Arabian Knights",
    "Underworld", "Sadie Thompson", "The Crowd", "A Ship Comes In", "Chang: A Drama of the Wilderness",
    "Glorious Betsy", "Sorrell and Son", "Speedy", "The Devil Dancer", "The Jazz Singer",
    "The Magic Flame", "The Noose", "The Patent Leather Kid", "The Private Life of Helen of Troy",
    "The Broadway Melody", "In Old Arizona", "The Patriot", "The Divine Lady", "Coquette",
    "The Bridge of San Luis Rey", "White Shadows in the South Seas", "Alibi", "Madame X",
    "Our Dancing Daughters", "The Valiant", "The Big House", "All Quiet on the Western Front",
    "The Divorcee", "Disraeli", "King of Jazz", "The Love Parade", "Anna Christie",
    "Bulldog Drummond", "Romance", "Condemned", "Hallelujah", "Hell's Angels",
    "Raffles", "Sally", "Sarah and Son", "Song of the Flame", "Street of Chance",
    "The Big Pond", "The Case of Sergeant Grischa", "The Devil's Holiday", "The Green Goddess",
    "The Rogue Song", "The Trespasser", "The Vagabond King", "Their Own Desire",
    "Cimarron", "Skippy", "A Free Soul", "Min and Bill", "Tabu: A Story of the South Seas",
    "The Dawn Patrol", "Morocco", "The Front Page"
]

# Function to fetch movie data in batches
def fetch_movie_data(movie_list):
    movie_data = []
    for title in movie_list:
        response = requests.get(f"{url}?t={title}{api_variable}")
        if response.status_code == 200:
            movie_data.append(response.json())
    return pd.DataFrame(movie_data)

# Fetch movie data in bulk
new_df = fetch_movie_data(movie_list)

# Apply transformations
new_df.fillna("Unknown", inplace=True)  # Handle missing values
new_df["Released"] = pd.to_datetime(new_df["Released"], errors="coerce").dt.strftime('%Y-%m-%d')  # Standardize date format
new_df = new_df[new_df["Poster"] != "N/A"]  # Filter unwanted entries (remove movies missing poster URLs)
new_df["imdbRating"] = pd.to_numeric(new_df["imdbRating"], errors="coerce")  # Convert IMDb ratings to float
new_df.drop_duplicates(subset=["Title"], inplace=True)  # Deduplicate based on title

In [9]:
# Transformation 1

# The pd.concat() function combines the existing dataset (data) 
# with new_df, appending new records while ignoring index conflicts 
# to maintain consistency.

# Merge with existing dataset
merged_df = pd.concat([data, new_df], ignore_index=True)

print(merged_df.head)

<bound method NDFrame.head of         id     type                                             title  \
0      1.0    Movie  Duck the Halls: A Mickey Mouse Christmas Special   
1      2.0    Movie                            Ernest Saves Christmas   
2      3.0    Movie                      Ice Age: A Mammoth Christmas   
3      4.0    Movie                        The Queen Family Singalong   
4      5.0  TV Show                             The Beatles: Get Back   
...    ...      ...                                               ...   
22912  NaN      NaN                                               NaN   
22913  NaN      NaN                                               NaN   
22914  NaN      NaN                                               NaN   
22915  NaN      NaN                                               NaN   
22916  NaN      NaN                                               NaN   

                                director  \
0      Alonso Ramirez Ramos, Dave Wasson   
1    

In [10]:
# Transformation 2

# The .fillna("Unknown", inplace=True) method replaces any missing values 
# in new_df with "Unknown", ensuring that blank fields don't cause issues 
# in downstream analysis.

# Handle Missing Values
new_df.fillna("Unknown", inplace=True)

In [11]:
# Transformation 3

# The pd.to_datetime() function converts the "Released" column to a proper datetime format, coercing 
# errors to NaT when needed, and then reformats it into the standard YYYY-MM-DD format for consistency.

# Standardize Date Format
new_df["Released"] = pd.to_datetime(new_df["Released"], errors="coerce").dt.strftime('%Y-%m-%d')

In [12]:
# Transformation 4

# The pd.to_numeric() function ensures that IMDb ratings are stored as numerical 
# values, allowing for proper calculations while gracefully handling any non-numeric 
# entries by converting them to NaN.

#Convert IMDb Ratings to Float
new_df["imdbRating"] = pd.to_numeric(new_df["imdbRating"], errors="coerce")

In [13]:
# Transformation 5

# The .drop_duplicates() function removes duplicate entries based 
# on the "Title" column, ensuring that each movie appears only once 
# in the dataset.

# Deduplicate Records Based on Title
new_df.drop_duplicates(subset=["Title"], inplace=True)

In [14]:
# Append new data to existing dataset
merged_df = pd.concat([data, new_df], ignore_index=True)

print(merged_df.head())

    id     type                                             title  \
0  1.0    Movie  Duck the Halls: A Mickey Mouse Christmas Special   
1  2.0    Movie                            Ernest Saves Christmas   
2  3.0    Movie                      Ice Age: A Mammoth Christmas   
3  4.0    Movie                        The Queen Family Singalong   
4  5.0  TV Show                             The Beatles: Get Back   

                            director  \
0  Alonso Ramirez Ramos, Dave Wasson   
1                        John Cherry   
2                       Karen Disher   
3                    Hamish Hamilton   
4                                NaN   

                                                cast        country  \
0  Chris Diamantopoulos, Tony Anselmo, Tress MacN...            NaN   
1           Jim Varney, Noelle Parker, Douglas Seale            NaN   
2  Raymond Albert Romano, John Leguizamo, Denis L...  United States   
3  Darren Criss, Adam Lambert, Derek Hough, Alexa...        

In [22]:
# Transformation 6

# The combination of the 2 df created new issues such as duplicates, naming structures, and missing values. 

# Standardize column names before merging
data.rename(columns=lambda x: x.strip().lower(), inplace=True)
new_df.rename(columns=lambda x: x.strip().lower(), inplace=True)

# Merge on 'title' to combine matching rows
combined_df = pd.merge(data, new_df, on="title", how="outer")

# Drop duplicate columns if they exist
columns_to_drop = ["unnamed: 0", "title_y"] if "title_y" in combined_df.columns else []
combined_df.drop(columns=columns_to_drop, inplace=True)

# Rename columns for clarity
combined_df.rename(columns={"title_x": "title", "imdbrating": "IMDb_Rating"}, inplace=True)

# Fill missing values
combined_df.fillna("Unknown", inplace=True)

# Display results
print(combined_df.head())

        id   type_x                          title  \
0   6997.0    Movie                     "Mixed Up"   
1   7427.0  TV Show          "The Paramedic Angel"   
2  13101.0    Movie                         #Alive   
3  13362.0    Movie  #AnneFrank - Parallel Stories   
4  13539.0    Movie              #FriendButMarried   

                    director_x  \
0                 Nishi Chawla   
1                      Unknown   
2                       Cho Il   
3  Sabina Fedeli, Anna Migotto   
4                Rako Prijanto   

                                                cast    country_x  \
0  UDAY KRISHNA, BETHANY RISHELL, LUCY BOND, SANJ...      Unknown   
1  Nate Reidnauer, Nikki Hrichak, Nina Randazzo, ...      Unknown   
2                           Yoo Ah-in, Park Shin-hye  South Korea   
3                        Helen Mirren, Gengher Gatti        Italy   
4  Adipati Dolken, Vanesha Prescilla, Rendi Jhon,...    Indonesia   

          date_added release_year  duration  \
0       

/var/folders/rs/8w5stb190rv90fd6dl0jz79r0000gn/T/ipykernel_68230/4191777250.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined_df.fillna("Unknown", inplace=True)


In [24]:
print(combined_df.columns)
print(combined_df.shape)

Index(['id', 'type_x', 'title', 'director_x', 'cast', 'country_x',
       'date_added', 'release_year', 'duration', 'listed_in', 'description',
       'unnamed: 0', 'imdb', 'rotten tomatoes', 'netflix', 'hulu',
       'prime video', 'disney+', 'rating_age', 'awards_x', 'nominations',
       'year_x', 'win_count', 'year_y', 'rated', 'released', 'runtime',
       'genre', 'director_y', 'writer', 'actors', 'plot', 'language',
       'country_y', 'awards_y', 'poster', 'ratings', 'metascore',
       'IMDb_Rating', 'imdbvotes', 'imdbid', 'type_y', 'dvd', 'boxoffice',
       'production', 'website', 'response', 'totalseasons'],
      dtype='object')
(22910, 48)


During the data wrangling process, several modifications were made to the dataset, including renaming columns, merging multiple sources, standardizing formats, replacing missing values, and handling duplicates to ensure consistency. These transformations raise ethical considerations regarding the accuracy and integrity of the data, particularly in cases where missing values were substituted or assumptions were made about categorical entries. For example, filling missing fields with "Unknown" might obscure critical nuances, especially in metadata such as awards or nominations. Since OMDb aggregates publicly available movie data, there are limited legal restrictions, as the dataset does not contain personally identifiable information. However, intellectual property concerns exist regarding film metadata and imagery. Ensuring ethical data usage requires verifying information accuracy, acknowledging assumptions, and minimizing unintended biases introduced during cleaning. To mitigate risks, maintaining transparency in all transformations, keeping a record of the original dataset, and cross-referencing sources for reliability are key to preserving data credibility and usability.

In [27]:
combined_df.to_excel("film_data_api.xlsx", index=False)